In [ ]:
# Libraries
from Bio import SeqIO
import pandas as pd
from collections import Counter

# -------------------------------
# Parameters
# -------------------------------
# Fixed order of the 20 amino acids (adjust if you prefer a different order)
amino_acid_order = list("MFGLVPSQCNITRYDKHWAE")
VALID_AMINO_ACIDS = set(amino_acid_order)

# -------------------------------
# Main functions
# -------------------------------
def count_and_distribute_all(file_path, amino_acid_order):
    """
    Reads ALL sequences from a protein FASTA file.
    - Counts only valid amino acids (20 canonical defined in amino_acid_order).
    - Calculates counts and normalized probabilities (per sequence).
    Returns:
        df_freq  : DataFrame of counts (frequencies)
        df_probs : DataFrame of normalized probabilities (normalized_distribution)
    """
    records = []
    for record in SeqIO.parse(file_path, "fasta"):
        sample_id = record.id
        seq = str(record.seq).upper()

        # Keep only valid amino acids (ignore X, B, Z, J, U, O, *, -, etc.)
        filtered_seq = "".join([aa for aa in seq if aa in VALID_AMINO_ACIDS])

        counts = Counter(filtered_seq)
        total_valid = sum(counts.values())

        # Counts in the fixed order
        ordered_counts = {aa: counts.get(aa, 0) for aa in amino_acid_order}

        # Normalized probabilities
        if total_valid > 0:
            ordered_probs = {aa: ordered_counts[aa] / total_valid for aa in amino_acid_order}
        else:
            ordered_probs = {aa: 0.0 for aa in amino_acid_order}

        records.append((
            sample_id,
            ordered_counts,
            ordered_probs,
            total_valid
        ))

    # DataFrame of counts
    df_freq = pd.DataFrame(
        {sample: counts for sample, counts, _, _ in records}
    ).T.reindex(columns=amino_acid_order).fillna(0).astype(int)
    df_freq.index.name = "Sequence_ID"

    # Add total count column
    totals = pd.Series({sample: total for sample, _, _, total in records}, name="Total_Count")
    df_freq = pd.concat([df_freq, totals], axis=1)

    # DataFrame of probabilities
    df_probs = pd.DataFrame(
        {sample: probs for sample, _, probs, _ in records}
    ).T.reindex(columns=amino_acid_order).fillna(0.0)
    df_probs.index.name = "Sequence_ID"

    return df_freq, df_probs


def save_excel_separately(df_freq, df_probs, output_prefix):
    """
    Saves two separate Excel files:
      - kmer_frequency_{prefix}.xlsx
      - probabilities_{prefix}.xlsx
    """
    freq_file = f"kmer_frequency_{output_prefix}.xlsx"
    probs_file = f"probabilities_{output_prefix}.xlsx"

    df_freq.to_excel(freq_file, index=True)
    df_probs.to_excel(probs_file, index=True)

    print(f"\nFiles successfully saved:")
    print(f" - {freq_file}")
    print(f" - {probs_file}")


# -------------------------------
# Example of use
# -------------------------------
if __name__ == "__main__":
    file_path = "Spike HCoV NL63.fasta"  # adjust to your FASTA file

    # Generate the tables
    df_frequencies, df_normalized = count_and_distribute_all(file_path, amino_acid_order)

    # Base prefix for saving files
    output_prefix = "Spike HCoV-NL63"

    # Save the tables separately
    save_excel_separately(df_frequencies, df_normalized, output_prefix)

    # Preview
    print("\nPreview - Frequencies:")
    print(df_frequencies.head())

    print("\nPreview - Normalized Distribution:")
    print(df_normalized.head())
